1_ Entender datos (pdf, href formula, href, Unnamed: 22, Unnamed: 23???)
2_ Comprobar columnas o filas duplicadas.
3_ Comprobar columnas o filas NaN.
4_ Eliminar columnas (pdf, href formula, href, Unnamed: 22, Unnamed: 23); lo que no dé valor.
5_ Cambiar nombres columnas.

In [1]:
# Libraries
import numpy as np
import pandas as pd
import re

Cargamos datos de formato CSV

In [2]:
attacks = pd.read_csv("../data/attacks.csv", encoding = "ISO-8859-1")

Visualizamos las primeras filas

In [3]:
attacks.head(3)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN


 Visualizo las últimas filas hola

In [4]:
attacks.tail(3)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25722,xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Visualizo muestra aleatoria

In [5]:
attacks.sample(3)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
22205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Decide a HYPOTHESIS (or hypotheses) with which you will clean the data

..............

ANALISIS EXPLORATORIO DEL DATAFRAME

In [6]:
attacks.shape

(25723, 24)

In [7]:
attacks.info

<bound method DataFrame.info of       Case Number         Date    Year        Type    Country  \
0      2018.06.25  25-Jun-2018  2018.0     Boating        USA   
1      2018.06.18  18-Jun-2018  2018.0  Unprovoked        USA   
2      2018.06.09  09-Jun-2018  2018.0     Invalid        USA   
3      2018.06.08  08-Jun-2018  2018.0  Unprovoked  AUSTRALIA   
4      2018.06.04  04-Jun-2018  2018.0    Provoked     MEXICO   
...           ...          ...     ...         ...        ...   
25718         NaN          NaN     NaN         NaN        NaN   
25719         NaN          NaN     NaN         NaN        NaN   
25720         NaN          NaN     NaN         NaN        NaN   
25721         NaN          NaN     NaN         NaN        NaN   
25722          xx          NaN     NaN         NaN        NaN   

                  Area                        Location     Activity  \
0           California     Oceanside, San Diego County     Paddling   
1              Georgia  St. Simon Island, Gly

In [8]:
attacks.isnull().sum()

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

In [9]:
attacks.describe()

,Year,original order
count,6300.000000,6309.000000
mean,1927.272381,3155.999683
std,281.116308,1821.396206
min,0.000000,2.000000
25%,1942.000000,1579.000000
50%,1977.000000,3156.000000
75%,2005.000000,4733.000000
max,2018.000000,6310.000000


In [10]:
attacks.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,6300.0,1927.272381,281.116308,0.0,1942.0,1977.0,2005.0,2018.0
original order,6309.0,3155.999683,1821.396206,2.0,1579.0,3156.0,4733.0,6310.0


COMPRUEBA DATOS NUMËRICOS Y CATEGÓRICOS

In [11]:
# DATOS NUMÉRICOS
numericas = attacks._get_numeric_data()
numericas.head(3)

,Year,original order
0,2018.0,6303.0
1,2018.0,6302.0
2,2018.0,6301.0


In [12]:
# DATOS CATEGÓRICOS
attacks.select_dtypes(include="object").head()

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,...,Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,...,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,NaN,NaN
1,2018.06.18,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,...,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,NaN,NaN
2,2018.06.09,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,...,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,NaN,NaN
3,2018.06.08,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,...,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,NaN,NaN
4,2018.06.04,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,...,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,NaN,NaN


RENOMBRO COLUMNAS

In [13]:
attacks_copia = attacks.copy()

In [14]:
columnas = list(attacks.columns)
columnas

['Case Number',
 'Date',
 'Year',
 'Type',
 'Country',
 'Area',
 'Location',
 'Activity',
 'Name',
 'Sex ',
 'Age',
 'Injury',
 'Fatal (Y/N)',
 'Time',
 'Species ',
 'Investigator or Source',
 'pdf',
 'href formula',
 'href',
 'Case Number.1',
 'Case Number.2',
 'original order',
 'Unnamed: 22',
 'Unnamed: 23']

In [15]:
diccio_nuevas = {columna: columna.replace(" ", "_").lower() for columna in columnas}

In [16]:
diccio_nuevas

{'Case Number': 'case_number',
 'Date': 'date',
 'Year': 'year',
 'Type': 'type',
 'Country': 'country',
 'Area': 'area',
 'Location': 'location',
 'Activity': 'activity',
 'Name': 'name',
 'Sex ': 'sex_',
 'Age': 'age',
 'Injury': 'injury',
 'Fatal (Y/N)': 'fatal_(y/n)',
 'Time': 'time',
 'Species ': 'species_',
 'Investigator or Source': 'investigator_or_source',
 'pdf': 'pdf',
 'href formula': 'href_formula',
 'href': 'href',
 'Case Number.1': 'case_number.1',
 'Case Number.2': 'case_number.2',
 'original order': 'original_order',
 'Unnamed: 22': 'unnamed:_22',
 'Unnamed: 23': 'unnamed:_23'}

In [17]:
attacks_renombrado = attacks.rename(columns=diccio_nuevas)
attacks_renombrado.sample()

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
22297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
attacks.rename(columns=diccio_nuevas, inplace=True)

In [19]:
attacks.head(2)

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN


In [20]:
attacks

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


EXPLORA DATOS POR COLUMNAS (apuntes)

In [21]:
attacks[attacks["unnamed:_23"].notna()]

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
4415,1952.03.30,30-Mar-1952,1952.0,Unprovoked,NETHERLANDS ANTILLES,Curacao,NaN,Went to aid of child being menaced by the shark,A.J. Eggink,M,...,"Bull shark, 2.7 m [9'] was captured & dragged ...","J. Randall, p.352 in Sharks & Survival; H.D. B...",1952.03.30-Eggink.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.03.30,1952.03.30,1888.0,NaN,Teramo
5840,1878.09.14.R,Reported 14-Sep-1878,1878.0,Provoked,USA,Connecticut,"Branford, New Haven County",Fishing,Captain Pattison,M,...,NaN,"St. Joseph Herald, 9/14/1878",1878.09.14.R-Pattison.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1878.09.14.R,1878.09.14.R,463.0,NaN,change filename


ELIMINO columnas que no me aportan valor.

In [22]:
attacks.drop(["pdf", "href_formula","href","unnamed:_22", "unnamed:_23"],axis=1,inplace=True)

In [23]:
attacks.head()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,case_number.1,case_number.2,original_order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04,2018.06.04,6299.0


Elimino filas con todo NaN

In [24]:
sharks = attacks.dropna(how="all", axis=0)

In [25]:
sharks.shape

(8703, 19)

LIMPIEZA columna; case_number

In [26]:
case_number = list(attacks.case_number.unique())

In [27]:
def quitar_a(x):
    if ".a" in str(x):
        return x.replace(".a", "")
    else:
        return x

In [28]:
sharks["case_number"] = sharks.case_number.apply(quitar_a)

<ipython-input-28-b9e4096ae751>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_a)


In [29]:
def quitar_b(x):
    if ".b" in str(x):
        return x.replace(".b", "")
    else:
        return x

In [30]:
sharks["case_number"] = sharks.case_number.apply(quitar_b)

<ipython-input-30-7a0d2429a33b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_b)


In [31]:
def quitar_c(x):
    if ".c" in str(x):
        return x.replace(".c", "")
    else:
        return x

In [32]:
sharks["case_number"] = sharks.case_number.apply(quitar_c)

<ipython-input-32-560dba157aa4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_c)


In [33]:
def quitar_d(x):
    if ".c" in str(x):
        return x.replace(".d", "")
    else:
        return x

In [34]:
sharks["case_number"] = sharks.case_number.apply(quitar_d)

<ipython-input-34-327631ea2d19>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_d)


In [35]:
def quitar_R(x):
    if ".R" in str(x):
        return x.replace(".R", "")
    else:
        return x

In [36]:
sharks["case_number"] = sharks.case_number.apply(quitar_R)

<ipython-input-36-46223fb203ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_R)


LIMPIEZA columna; date

In [37]:
def date(x):
    
    return x.extract(r"(\w+-\d+)")

In [38]:
sharks['newcol'] = sharks['date'].str.extract(r"(\w+-\d+)")

<ipython-input-38-65ad4d2e00fd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['newcol'] = sharks['date'].str.extract(r"(\w+-\d+)")


In [39]:
sharks.sample(3)

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,case_number.1,case_number.2,original_order,newcol
7048,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
769,2012.07.07,07-Jul-2012,2012.0,Boating,USA,California,"Pleasure Point, Santa Cruz County",Kayaking,M.C.,M,52,"No injury, kayak bitten",N,08h30,14' to 18'shark,R. Collier,2012.07.07.a,2012.07.07.a,5534.0,Jul-2012
2242,1997.07.21,21-Jul-1997,1997.0,Unprovoked,SOUTH AFRICA,Eastern Cape Province,"Breezy Point, Ntlonyana",Surfing,Mark Penches,M,25,FATAL,Y,12h45,4.5 m white shark,"A. Gifford, GSAF, G. Cliff, NSB",1997.07.21,1997.07.21,4061.0,Jul-1997


In [47]:
dates = list(sharks.date)
print(dates[0:5])

['Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018']


In [48]:
newcoll= list(sharks.newcol)
print(newcoll[0:5])

['Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018']


In [49]:
sharks.date=newcoll

In [50]:
print(dates[0:5])

['Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018']


In [51]:
sharks.sample()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,case_number.1,case_number.2,original_order,newcol
8594,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
sharks.drop(["newcol"],axis=1, inplace=True)

/home/jesus_chao/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [53]:
sharks.sample()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,case_number.1,case_number.2,original_order
5939,1864.09.18,Sep-1864,1864.0,Provoked,FRANCE,Alpes Maritime,Antibes,Dragging a shark,fisherman,M,NaN,Knee bitten PROVOKED INCIDENT,N,NaN,1.5 m shark,"C. Moore, GSAF",1864.09.18.R,1864.09.18.R,364.0


LIMPIEZA columna; year

In [54]:
def year_(x):
    if ".0" in str(x):
        return str(x).replace(".0", "")
    else:
        return x

In [55]:
sharks["year"] = sharks.year.apply(year_)

<ipython-input-55-34703d824513>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["year"] = sharks.year.apply(year_)


In [56]:
print(sharks.year[0:5])

0    2018
1    2018
2    2018
3    2018
4    2018
Name: year, dtype: object


LIMPÎEZA Columna type.

In [57]:
sharks.type.unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

In [58]:
def type_(x):
    if "Unprovoked" in str(x):
        return x.replace("Unprovoked", "Accidental")
    else:
        return x

In [59]:
sharks["type"] = sharks.type.apply(type_)

<ipython-input-59-e5a48b295325>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["type"] = sharks.type.apply(type_)


In [60]:
multi_p = sharks[(sharks.type == "Boatomg")]

In [61]:
multi_p

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,case_number.1,case_number.2,original_order
1152,2009.03.01,Mar-2009,2009,Boatomg,NEW ZEALAND,North Island,Taranaki,Fishing,"boat, occupants: Boyd Rutherford & Hamish Roper",M,NaN,"No injury to occupants, shark bit propeller",N,NaN,NaN,"Taranaki Daily News, 3/3/2009",2009.03.01.b,2009.03.01.b,5151.0


In [62]:
def type_(x):
    if "Boatomg" in str(x):
        return x.replace("Boatomg", "Boat")
    else:
        return x

In [63]:
sharks["type"] = sharks.type.apply(type_)

<ipython-input-63-e5a48b295325>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["type"] = sharks.type.apply(type_)


In [64]:
def type_(x):
    if "Invalid" in str(x):
        return x.replace("Invalid", "Injured/Fatal")
    else:
        return x

In [65]:
sharks["type"] = sharks.type.apply(type_)

<ipython-input-65-e5a48b295325>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["type"] = sharks.type.apply(type_)


In [66]:
sharks.type.unique()

array(['Boating', 'Accidental', 'Injured/Fatal', 'Provoked',
       'Questionable', 'Sea Disaster', nan, 'Boat'], dtype=object)

LIMPIEZA Pais, área y localización

In [67]:
countries = list(sharks.country.unique())

In [68]:
len(countries)

213

In [69]:
print(countries)

['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA', 'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA', 'ECUADOR', 'MALAYSIA', 'LIBYA', nan, 'CUBA', 'MAURITIUS', 'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS', 'JAPAN', 'EGYPT', 'ST HELENA, British overseas territory', 'COMOROS', 'REUNION', 'FRENCH POLYNESIA', 'UNITED KINGDOM', 'UNITED ARAB EMIRATES', 'PHILIPPINES', 'INDONESIA', 'CHINA', 'COLUMBIA', 'CAPE VERDE', 'Fiji', 'DOMINICAN REPUBLIC', 'CAYMAN ISLANDS', 'ARUBA', 'MOZAMBIQUE', 'FIJI', 'PUERTO RICO', 'ITALY', 'ATLANTIC OCEAN', 'GREECE', 'ST. MARTIN', 'FRANCE', 'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL', 'DIEGO GARCIA', 'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES', 'GUAM', 'SEYCHELLES', 'BELIZE', 'NIGERIA', 'TONGA', 'SCOTLAND', 'CANADA', 'CROATIA', 'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA', 'RUSSIA', 'TURKS & CAICOS', 'UNITED ARAB EMIRATES (UAE)', 'AZORES', 'SOUTH KOREA', 'MALTA', 'VIETNAM', 'MADAGASCAR', 'PANAMA', 'SOMALIA', 

Ponemos todos los países en mayúsculas.

In [70]:
def upper_c(x):
    return str(x).upper()

In [71]:
sharks["country"] = sharks.country.apply(upper_c)

<ipython-input-71-51f75b38b595>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["country"] = sharks.country.apply(upper_c)


Reemplazar ENGLAND por UNITED KINGDOM

In [72]:
def england_(x):
    if "ENGLAND" in str(x):
        return x.replace("ENGLAND", "UNITED KINGDOM")
    else:
        return x

In [73]:
sharks["country"] = sharks.country.apply(england_)

<ipython-input-73-9a5641fac9da>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["country"] = sharks.country.apply(england_)


ELIMINO signo interrogación

In [74]:
def question_(x):
    if "?" in str(x):
        return x.replace("?", "")
    else:
        return x

In [75]:
sharks["country"] = sharks.country.apply(question_)

<ipython-input-75-1d540e1a7bd3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["country"] = sharks.country.apply(question_)


In [76]:
sharks.country.unique()

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'UNITED KINGDOM',
       'SOUTH AFRICA', 'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS',
       'NEW CALEDONIA', 'ECUADOR', 'MALAYSIA', 'LIBYA', 'NAN', 'CUBA',
       'MAURITIUS', 'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS',
       'JAPAN', 'EGYPT', 'ST HELENA, BRITISH OVERSEAS TERRITORY',
       'COMOROS', 'REUNION', 'FRENCH POLYNESIA', 'UNITED ARAB EMIRATES',
       'PHILIPPINES', 'INDONESIA', 'CHINA', 'COLUMBIA', 'CAPE VERDE',
       'FIJI', 'DOMINICAN REPUBLIC', 'CAYMAN ISLANDS', 'ARUBA',
       'MOZAMBIQUE', 'PUERTO RICO', 'ITALY', 'ATLANTIC OCEAN', 'GREECE',
       'ST. MARTIN', 'FRANCE', 'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO',
       'KIRIBATI', 'ISRAEL', 'DIEGO GARCIA', 'TAIWAN', 'JAMAICA',
       'PALESTINIAN TERRITORIES', 'GUAM', 'SEYCHELLES', 'BELIZE',
       'NIGERIA', 'TONGA', 'SCOTLAND', 'CANADA', 'CROATIA',
       'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA', 'RUSSIA',
       'TURKS & CAICOS', 'UNITED ARAB EMIRATES (

In [77]:
areas = list(sharks.area.unique())

In [78]:
len(areas)

826

In [79]:
areas

['California',
 'Georgia',
 'Hawaii',
 'New South Wales',
 'Colima',
 'Pernambuco',
 'Florida',
 'Queensland',
 'South Carolina',
 'Cornwall',
 'Westerm Australia',
 'Eastern Cape Province',
 'Hua Hin',
 'Cocos Island',
 'Western Australia',
 'Alifu Alifu Atoll',
 'Western Cape Province',
 'New Providence',
 'Alagoas',
 nan,
 'Victoria',
 'KwaZulu-Natal',
 'Boi Island, Victoria',
 'Galapagos Islands',
 'Fernando de Noronha',
 'Sepang',
 'Holquin Province',
 'Pamplemousses ',
 'South Australia',
 'North Island',
 'New York',
 'Canary Islands',
 ' Upolu Island',
 'Shizuoka Prefecture',
 'Texas',
 'Castellón',
 'Massachusetts',
 'Red Sea Protectorate',
 'New Providence District',
 '40 miles off Grand Bahama Island',
 'Ascension Island',
 'New Jersey',
 'Majorca',
 'Washington',
 'Tabasco',
 'Anjouan',
 'Ibiza Island',
 'Marquesas',
 'South Devon',
 'New Providence ',
 'Sharjah, ',
 'Baja California Sur',
 'Saint-Leu',
 'South Island',
 'Luzon Island',
 'Great Exuma',
 'Saint-Andre',
 'Bim

In [80]:
locations = list(sharks.location.unique())

In [81]:
len(locations)

4109

In [82]:
locations

['Oceanside, San Diego County',
 'St. Simon Island, Glynn County',
 'Habush, Oahu',
 'Arrawarra Headland',
 'La Ticla',
 'Flat Rock, Ballina',
 'Piedade Beach, Recife',
 'Lighhouse Point Park, Ponce Inlet, Volusia County',
 'Cocoa Beach, Brevard  County',
 'Daytona Beach, Volusia County',
 'Cairns Aquarium',
 'Isle of Palms,  Charleston County',
 'Hilton Head Island, Beaufort County',
 "Off Land's End",
 'Dugong Bay',
 'Nahoon Beach, East London',
 'Sharpes Beach, Ballina',
 'Sai Noi Beach',
 'Manuelita',
 'Gearys Beach',
 'Lennox Head',
 'Surfers Point, Prevelly',
 'South Point, Gracetown',
 'Madoogali',
 'Robberg Beach, Plettenberg Bay',
 'Shipwreck\x92s Beach, Keoneloa Bay, Kauai',
 'Piedade',
 'Lefthanders, Margaret River Area',
 'Cobblestones, Margaret River Area',
 'Nirvana Beach',
 'Praia de Sauaçuhy, Maceió',
 'Magenta Beach, Noumea',
 'Bimini',
 'St. Francis Bay',
 'Kukio Beach',
 "Waterman's Bay",
 'Lorne',
 'Winkipop',
 'Durban',
 'Little Congwong Beach, La Perouse ',
 'West

In [ ]:
location = sharks[["country", "area", "location"]]

In [ ]:
multi_USA = location[(sharks.country == "USA")]

In [ ]:
multi_USA

OPERA CON COLUMNAS (SELECCIONA, ELIMINA, Valores únicos ...)

Filtramos los datos por el valor de las columnas

In [ ]:
attacks.area.unique()

BORRAR FILAS Y COLUMNAS. Reset_index y set_index

APPLY!!

Df .loc e .iloc
Seleccionar filas y columnas en Pandas mediante posición con iloc
Seleccionar filas y columnas en Pandas en base a etiquetas con loc

Ordenamos un dataframe
Buscamos valores nulos
Borramos los valores nulos
rellenar los datos que faltan
 Borramos los duplicados

 Agregación de datos
 GroupBy

EXPLORE the DATA and write down what you have found
you can use: df.describe(), df["column"], etc.

DRAW at least 2 GRAPHS that are insightful.

Use at least 5 data cleaning techniques inside a file named clean.ipynb
null values, columns drop, duplicated data, string manipulation, apply fn, categorize, regex, etc.

Show data that validates the conclusions based on your hypoteses in a file named analysis.ipynb

Build a compelling story-telling around your findings. Think of your stakeholders and convince them with your conclusions! (Some slides with few text and pretty plots are normally useful)